# Import & Load CASEN health data

### Config

In [35]:
import sys,os
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

In [36]:
remote_path = 'http://pacha.datawheel.us/'
local_path = '../data/'

In [37]:
import postgres #from local file postgres.py
from commons import inline_table_xml, inline_dimension_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load data

In [38]:
download_zip_file(remote_path,
                  local_path,
                  "healthsystem.zip")

extract_zip_file(local_path, "temp.zip")

Downloading... http://pacha.datawheel.us/healthsystem.zip
Unzipping... ../data/temp.zip


True

In [41]:
df1 = pd.read_csv(os.path.join(local_path, "healthsystem.csv"),low_memory=True)

In [42]:
df1 = df1.astype({'year':'int','comuna_datachile_id':'int','health_system_id':'int'})
list(df1)

['year', 'comuna_datachile_id', 'health_system_id', 'expc', 'expr']

In [43]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df1, 'health', 'fact_health_system_casen')

DROP TABLE IF EXISTS health.fact_health_system_casen;
CREATE TABLE "health"."fact_health_system_casen" (
"year" INTEGER,
  "comuna_datachile_id" INTEGER,
  "health_system_id" INTEGER,
  "expc" INTEGER,
  "expr" INTEGER
)
COPY "health"."fact_health_system_casen" ("year","comuna_datachile_id","health_system_id","expc","expr") FROM STDIN WITH CSV HEADER DELIMITER ',';


### Dims

In [50]:
d2 = pd.read_csv(os.path.join(local_path, "healthsystem_ids.csv"),low_memory=True)
d2 = d2.fillna(0)
d2 = d2.astype({'health_system_id':'int'})
print (inline_table_xml(d2, 'health_system', 'health_system_id', 'health_system'))


<InlineTable alias="health_system">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">Fonasa A</Value>
      <Value column="es_description">Fonasa A</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Fonasa B</Value>
      <Value column="es_description">Fonasa B</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">Fonasa C</Value>
      <Value column="es_description">Fonasa C</Value>
    </Row>
  <Row>
      <Value column="id">4</Value>
      <Value column="description">Fonasa D</Value>
      <Value column="es_description">Fonasa D</Value>
    </Row>
  <Row>
      <Value column="id">5</Value>
      <Value column="description">Fonasa (no sabe grupo)</Value>
      <Value column="es_descr